# 학교 500m 이내 CCTV 계산
- data 
    - 나이스_서울_전체_위경도_학교기본정보_Ver2.csv
    - 서울시 (안심이) CCTV 설치 현황_위경도.csv
- data(output)
    - 학교500m_cctv_개수_Ver2.csv

In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings(action='ignore')

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist
from sklearn.preprocessing import StandardScaler
from haversine import haversine
from tqdm import tqdm

In [25]:
# 나이스 학교 데이터 불러오기
school_loc_data = pd.read_csv("D:\MultiCampus\DATAexam\융복합\폭력분석\data\나이스_서울_전체_위경도_학교기본정보_Ver2.csv", index_col=0)
school_loc_data.head(1)

,시도교육청코드,시도교육청명,표준학교코드,학교명,영문학교명,학교종류명,소재지명,시군구,관할조직명,설립명,...,산업체특별학급존재여부,고등학교일반실업구분명,특수목적고등학교계열명,입시전후기구분명,주야구분명,설립일자,개교기념일,수정일,위도,경도
0,B10,서울특별시교육청,7010057,가락고등학교,Garak High School,고등학교,서울특별시,송파구,서울특별시교육청,공립,...,N,일반계,NaN,후기,주간,19881223,19890428,20230115,37.501076,127.116426


In [26]:
cctv_loc_data = pd.read_csv("D:\MultiCampus\DATAexam\융복합\폭력분석\data\서울시 (안심이) CCTV 설치 현황_위경도.csv", index_col=0)
cctv_loc_data.head()

,시군구,안심 주소,위도,경도,CCTV 수량
0,중랑구,상봉1동 산4-1 (봉수대공원그네앞)_C-SB01-0016,37.6070,127.0913,1
1,중랑구,상봉1동 99(고정2)_C-SB01-0028-B,37.5977,127.0884,1
2,중랑구,상봉1동 99(고정1)_C-SB01-0028-A,37.5977,127.0884,1
3,중랑구,상봉1동 84-34(고정4)_922,37.5942,127.0930,1
4,중랑구,상봉1동 84-34(고정3)_922,37.5942,127.0930,1


In [27]:
cctv_loc_data.rename(columns={'CCTV 수량': 'CCTV_수량'}, inplace=True)
cctv_loc_data.head()

,시군구,안심 주소,위도,경도,CCTV_수량
0,중랑구,상봉1동 산4-1 (봉수대공원그네앞)_C-SB01-0016,37.6070,127.0913,1
1,중랑구,상봉1동 99(고정2)_C-SB01-0028-B,37.5977,127.0884,1
2,중랑구,상봉1동 99(고정1)_C-SB01-0028-A,37.5977,127.0884,1
3,중랑구,상봉1동 84-34(고정4)_922,37.5942,127.0930,1
4,중랑구,상봉1동 84-34(고정3)_922,37.5942,127.0930,1


In [50]:
# CCTV수 구하는 함수 
def close_cctv(school_loc_data, cctv_loc_data): 
    schoolloc, cctvloc, cctv_cnt = [], [], []
    for col in tqdm(school_loc_data.itertuples()): 
        schoolloc = [col.위도, col.경도]
        distance = []
        for col2 in cctv_loc_data.itertuples(): 
            cctvloc = [col2.위도, col2.경도]
            distance.append(haversine(schoolloc, cctvloc, unit = 'm'))
            
        distance = [d for d in distance if d <= 500]
        cctv_cnt.append(len(distance)*col2.CCTV_수량)
    return cctv_cnt

In [51]:
school_loc_data['500m_inner_cctv'] = close_cctv(school_loc_data, cctv_loc_data)
school_loc_data

320it [01:36,  3.31it/s]


,시도교육청코드,시도교육청명,표준학교코드,학교명,영문학교명,학교종류명,소재지명,시군구,관할조직명,설립명,...,고등학교일반실업구분명,특수목적고등학교계열명,입시전후기구분명,주야구분명,설립일자,개교기념일,수정일,위도,경도,500m_inner_cctv
0,B10,서울특별시교육청,7010057,가락고등학교,Garak High School,고등학교,서울특별시,송파구,서울특별시교육청,공립,...,일반계,NaN,후기,주간,19881223,19890428,20230115,37.501076,127.116426,102
1,B10,서울특별시교육청,7011169,가재울고등학교,Gajaeul High School,고등학교,서울특별시,서대문구,서울특별시교육청,공립,...,일반계,NaN,후기,주간,20130301,20130501,20230115,37.574364,126.909530,137
2,B10,서울특별시교육청,7010117,강동고등학교,Kangdong High School,고등학교,서울특별시,강동구,서울특별시교육청,사립,...,일반계,NaN,후기,주간,19841217,19850908,20230115,37.549917,127.160668,51
3,B10,서울특별시교육청,7010118,강서고등학교,Gangseo High School,고등학교,서울특별시,양천구,서울특별시교육청,사립,...,일반계,NaN,후기,주간,19831101,19831101,20230115,37.536316,126.865771,299
4,B10,서울특별시교육청,7010270,강서공업고등학교,Gangseo Technical High School,고등학교,서울특별시,강서구,서울특별시교육청,공립,...,전문계,NaN,전기,주간,19940105,19940105,20230115,37.576034,126.816182,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,B10,서울특별시교육청,7010766,효문고등학교,Hyomun High School,고등학교,서울특별시,도봉구,서울특별시교육청,공립,...,일반계,NaN,후기,주간,20051229,20060605,20230115,37.652699,127.018400,109
316,B10,서울특별시교육청,7010292,휘경공업고등학교,Hwigyeong Technical High School,고등학교,서울특별시,동대문구,서울특별시교육청,공립,...,전문계,NaN,전기,주간,19971220,19980702,20230115,37.585153,127.069451,99
317,B10,서울특별시교육청,7010267,휘경여자고등학교,Hwikyung Girls` High School,고등학교,서울특별시,동대문구,서울특별시교육청,사립,...,일반계,NaN,후기,주간,19750303,19750519,20230115,37.581901,127.067434,136
318,B10,서울특별시교육청,7010268,휘문고등학교,Whimoon High School,고등학교,서울특별시,강남구,서울특별시교육청,사립,...,일반계,NaN,전기,주간,19060501,19060501,20230115,37.505289,127.061994,191


In [53]:
school_loc_data.to_csv("D:\MultiCampus\DATAexam\융복합\폭력분석\data\학교500m_cctv_개수_Ver2.csv", index=False, encoding='utf-8-sig')